In [1]:
#start

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
16,application_1726802887048_0018,pyspark,idle,Link,Link,user_n-renukumar_rajendran_draup_com,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sc.addPyFile("s3://jobs-emr-etl-config/jobs_run/prod/cc/ML_ETL_Config.py")
from ML_ETL_Config import *
utils_module = MlEtlUtils(spark, sc)
to_import = [name for name in dir(utils_module) if not (name.startswith('_') | (name=="spark") | (name=="sc"))]
globals().update({name: getattr(utils_module, name) for name in to_import})
from pyspark.sql.functions import *
from pyspark.sql import Window

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

404
404
/usr/local/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gateway.draup.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/local/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gateway.draup.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(

# parquet input

In [3]:

pce = spark.read.format("delta").load("s3://etl-datahouse/prod/profiles/pce/pce_calc/pce_snapshot/delta").select('mvp_company_name','currentPositions','pastPositions','publicLink', 'fullName', 'gender','ethnicity','functional_workload','business_function','msa','experience_in_current_company','experience_in_current_job_role','years_of_experience','languages').selectExpr('*','currentPositions[0].companyName as curr_company','currentPositions[0].company_industry as current_industry','currentPositions[0].job_title_lemmatized as current_job_title','currentPositions.job_role[0] as current_job_role','currentPositions[0].msa as current_msa','currentPositions[0].synon_location as current_synon_location','currentPositions[0].i18nStartDate as current_StartDate','currentPositions[0].i18nEndDate as current_EndDate','currentPositions[0].startDateYear as startDateYear','currentPositions[0].endDateYear as endDateYear','pastPositions[0].companyName as past_company','pastPositions[0].company_industry as past_industry','pastPositions[0].experience_years as past_company_experience','pastPositions[0].i18nStartDate as past_startDateYear','pastPositions[0].i18nEndDate as past_endDateYear','pastPositions[0].job_title_lemmatized as past_title','pastPositions[0].msa as past_msa','pastPositions[0].synon_location as past_synon_location','currentPositions[0].company_type as curr_company_type','pastPositions[0].company_type as past_company_type').drop('currentPositions','pastPositions','past_data','deleted')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
pce_data = spark.read.parquet('s3://draup-command-center/temp/cc/Renukumar_Draup/karthika/pce_profile_past_role_data_16-09-2024')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
pce_data.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- mvp_company_name: string (nullable = true)
 |-- publicLink: string (nullable = true)
 |-- fullName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- ethnicity: struct (nullable = true)
 |    |-- category: string (nullable = true)
 |    |-- probability: double (nullable = true)
 |-- functional_workload: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- business_function: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- msa: string (nullable = true)
 |-- experience_in_current_company: double (nullable = true)
 |-- experience_in_current_job_role: double (nullable = true)
 |-- years_of_experience: double (nullable = true)
 |-- languages: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- languageName: string (nullable = true)
 |    |    |-- proficiency: string (nullable = true)
 |-- country: string (nullable = true)
 |-- curr_company: string (nullable = true)
 |-- current

### location input file

In [9]:
loc_df = spark.read.csv('s3://draup-command-center/temp/cc/Renukumar_Draup/karthika/Input/msa_pce_input_20_09_24.csv',header = True).withColumnRenamed('msa','msa_data')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
loc_df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['msa_data', 'Country', 'Region']

In [11]:
loc_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

109

In [13]:
join_df =pce_data.join(loc_df,pce_data['msa'] == loc_df['msa_data'],'inner').drop("msa_data")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
join_df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['mvp_company_name', 'publicLink', 'fullName', 'gender', 'ethnicity', 'functional_workload', 'business_function', 'msa', 'experience_in_current_company', 'experience_in_current_job_role', 'years_of_experience', 'languages', 'country', 'curr_company', 'current_industry', 'current_job_title', 'current_job_role', 'current_msa', 'current_synon_location', 'current_StartDate', 'current_EndDate', 'startDateYear', 'endDateYear', 'past_company', 'past_industry', 'past_company_experience', 'past_startDateYear', 'past_endDateYear', 'past_title', 'past_msa', 'past_synon_location', 'past_job_role', 'Country', 'Region']

### Role input file

In [24]:
role_df = spark.read.csv('s3://draup-command-center/temp/cc/Renukumar_Draup/karthika/Input/role_pce_input_file_20_09_24.csv',header = True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
role_df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Platform Roles']

In [26]:
role_df.show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------------+
|Platform Roles                 |
+-------------------------------+
|Software Development Engineer  |
|Applications Developer/Engineer|
+-------------------------------+

In [27]:
explode_df = join_df.withColumn('Roles',explode(col('current_job_role')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
role_df_tag = explode_df.join(role_df, explode_df['Roles'] == role_df['Platform Roles'],'inner').drop('Roles')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
role_df_tag.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

162094

In [30]:
role_df_tag.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['mvp_company_name', 'publicLink', 'fullName', 'gender', 'ethnicity', 'functional_workload', 'business_function', 'msa', 'experience_in_current_company', 'experience_in_current_job_role', 'years_of_experience', 'languages', 'country', 'curr_company', 'current_industry', 'current_job_title', 'current_job_role', 'current_msa', 'current_synon_location', 'current_StartDate', 'current_EndDate', 'startDateYear', 'endDateYear', 'past_company', 'past_industry', 'past_company_experience', 'past_startDateYear', 'past_endDateYear', 'past_title', 'past_msa', 'past_synon_location', 'past_job_role', 'Country', 'Region', 'Platform Roles']

In [32]:
from io import StringIO # python3; python2: BytesIO import boto3
import boto3


bucket = 'draup-command-center' # already created on S3
csv_buffer = StringIO()

res2=role_df_tag.toPandas()
res2.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'temp/cc/Renukumar_Draup/karthika/output/pce_profile_datas_20-09-2024.csv').put(Body=csv_buffer.getvalue())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': 'XMXN4DT71PJ4KQ9S', 'HostId': 'sW1dnOKEIYy/7fiDAnedmXzZSVTwqBmU0KO2k44qyqgLrL3sqnN1KAAoCBQGUmIKJLrwTfNFDf0EtF+oor6Yj4I1nwQCIClm', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'sW1dnOKEIYy/7fiDAnedmXzZSVTwqBmU0KO2k44qyqgLrL3sqnN1KAAoCBQGUmIKJLrwTfNFDf0EtF+oor6Yj4I1nwQCIClm', 'x-amz-request-id': 'XMXN4DT71PJ4KQ9S', 'date': 'Fri, 20 Sep 2024 07:01:35 GMT', 'x-amz-expiration': 'expiry-date="Thu, 26 Sep 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'x-amz-server-side-encryption': 'AES256', 'etag': '"e8d1e5e900f392a8d9d7bedd6f5a7bce"', 'content-length': '0', 'server': 'AmazonS3'}, 'RetryAttempts': 1}, 'Expiration': 'expiry-date="Thu, 26 Sep 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'ETag': '"e8d1e5e900f392a8d9d7bedd6f5a7bce"', 'ServerSideEncryption': 'AES256'}

In [ ]:
Advertising Executive

In [ ]:
filtered_df = pce_data.filter(pce_data.curr_title.contains("Data Science"))

### Role,msa count

In [29]:
group_df =role_df_tag.groupBy('Roles','profile_msa','gender').count().orderBy(desc('count'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
from io import StringIO # python3; python2: BytesIO import boto3
import boto3


bucket = 'draup-command-center' # already created on S3
csv_buffer = StringIO()

res2=group_df.toPandas()
res2.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'temp/cc/Renukumar_Draup/karthika/output/pce_role_msa_gender_23-07-2024.csv').put(Body=csv_buffer.getvalue())

### Role,country count

In [ ]:
role_df_tag1 = role_df_tag.drop_duplicates(subset=['Role', 'publicLink', 'country_data'])

In [17]:
group_df1 =role_df_tag1.groupBy('Role','country_data').count().orderBy(desc('count'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
from io import StringIO # python3; python2: BytesIO import boto3
import boto3


bucket = 'draup-command-center' # already created on S3
csv_buffer = StringIO()

res2=group_df1.toPandas()
res2.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'temp/cc/Renukumar_Draup/karthika/output/pce_role_country_19-07-2024.csv').put(Body=csv_buffer.getvalue())

### Role,msa,Ethnicity count

In [18]:
group_df2 = role_df_tag.groupBy('Role','profile_msa','ethnicity').count().orderBy('count', ascending = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
from io import StringIO # python3; python2: BytesIO import boto3
import boto3


bucket = 'draup-command-center' # already created on S3
csv_buffer = StringIO()

res2=group_df2.toPandas()
res2.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'temp/cc/Renukumar_Draup/karthika/output/pce_role_YOF_19-07-2024.csv').put(Body=csv_buffer.getvalue())

### Role,Country,YOF count

In [19]:
role_df_tag1 = role_df_tag.drop_duplicates(subset=['Role', 'publicLink', 'country_data'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
group_df3 =role_df_tag1.groupBy('Role','country_data','years_of_experience').count().orderBy(desc('count'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
from io import StringIO # python3; python2: BytesIO import boto3
import boto3


bucket = 'draup-command-center' # already created on S3
csv_buffer = StringIO()

res2=group_df3.toPandas()
res2.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'temp/cc/Renukumar_Draup/karthika/output/pce_role_YOF_19-07-2024.csv').put(Body=csv_buffer.getvalue())

### Role, msa,core_skills count

In [22]:
role_df_tag = role_df_tag.withColumn('core_skills_data',explode(col('core_skills')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
group_df2 = role_df_tag.groupBy('Role','core_skills_data',').count().orderBy('count', ascending = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
from io import StringIO # python3; python2: BytesIO import boto3
import boto3


bucket = 'draup-command-center' # already created on S3
csv_buffer = StringIO()

res2=group_df2.toPandas()
res2.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'temp/cc/Renukumar_Draup/karthika/output/pce_role_msa_skills_19-07-2024.csv').put(Body=csv_buffer.getvalue())

### Role,country,core_skills count

In [25]:
role_df_tag = role_df_tag.drop_duplicates(subset=['Role', 'publicLink', 'country_data'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
skills_df = role_df_tag.withColumn('core_skills_data',explode(col('core_skills')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
group_df2 = skills_df.groupBy('Role','country_data','core_skills_data').count().orderBy('count', ascending = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
from io import StringIO # python3; python2: BytesIO import boto3
import boto3


bucket = 'draup-command-center' # already created on S3
csv_buffer = StringIO()

res2=final_df.toPandas()
res2.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'temp/cc/Renukumar_Draup/karthika/output/pce_role_country_skills_19-07-2024.csv').put(Body=csv_buffer.getvalue())

### Groupby Role,msa,skills,YOE

In [41]:
skills_df = role_df_tag.withColumn('core_skills_data',explode(col('core_skills')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
YE_df = skills_df.withColumn(
    'years_Experience',
    F.when((col('years_of_experience') >= 0) & (col('years_of_experience') < 3), '0-3')
     .when((col('years_of_experience') >= 3) & (col('years_of_experience') < 20), '3-20'))                

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
group_df2 = YE_df.groupBy('Role','country_data','core_skills_data').count().orderBy('count', ascending = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Top 20 profile

In [32]:
from pyspark.sql import Window
from pyspark.sql.functions import row_number
 
# Create a window specification
windowSpec = Window.partitionBy('Job role','Location','core_skills_data','years_Experience').orderBy(F.desc('count'))
 
# Add a row number column to each partition
grp_df10_with_row_num = group_df.withColumn('row_num', row_number().over(windowSpec))
 
# Filter to get only the top 10 rows per partition
final_df = grp_df10_with_row_num.filter(col('row_num') <= 20).drop('row_num')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
from io import StringIO # python3; python2: BytesIO import boto3
import boto3


bucket = 'draup-command-center' # already created on S3
csv_buffer = StringIO()

res2=final_df.toPandas()
res2.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'temp/cc/Renukumar_Draup/karthika/output/pce_role_msa_skills_MVP_yearExpr_20 top_19-07-2024.csv').put(Body=csv_buffer.getvalue())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': 'YWH0X9H1C1CFN78V', 'HostId': 'qFjF6GP7BUhPo6ZIhiHT8Ubc1P/+Zjy/CdSuZRs3D1yK8hWQnHxuuyeIJotlJJfKkaxOQXNDwLI=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'qFjF6GP7BUhPo6ZIhiHT8Ubc1P/+Zjy/CdSuZRs3D1yK8hWQnHxuuyeIJotlJJfKkaxOQXNDwLI=', 'x-amz-request-id': 'YWH0X9H1C1CFN78V', 'date': 'Fri, 19 Jul 2024 06:22:34 GMT', 'x-amz-expiration': 'expiry-date="Thu, 25 Jul 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'x-amz-server-side-encryption': 'AES256', 'etag': '"2250a536ee112b4205576f2c97e6ce06"', 'server': 'AmazonS3', 'content-length': '0'}, 'RetryAttempts': 1}, 'Expiration': 'expiry-date="Thu, 25 Jul 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'ETag': '"2250a536ee112b4205576f2c97e6ce06"', 'ServerSideEncryption': 'AES256'}

### Role, MVP company count

In [30]:
group_df1 = role_df_tag.groupBy('Role','mvp_company_name').count().orderBy(col('count').desc())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
from io import StringIO # python3; python2: BytesIO import boto3
import boto3


bucket = 'draup-command-center' # already created on S3
csv_buffer = StringIO()

res2=group_df1.toPandas()
res2.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'temp/cc/Renukumar_Draup/karthika/output/pce_role_mvp_19-07-2024.csv').put(Body=csv_buffer.getvalue())

### Groupby MVP company,msa

In [ ]:
group_df3 = role_df_tag.groupBy('mvp_company_name','profile_msa').count().orderBy('count', ascending = False)

In [ ]:
from io import StringIO # python3; python2: BytesIO import boto3
import boto3


bucket = 'draup-command-center' # already created on S3
csv_buffer = StringIO()

res2=group_df3.toPandas()
res2.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'temp/cc/Renukumar_Draup/karthika/output/pce_MVPcompany_msa_18-07-2024.csv').put(Body=csv_buffer.getvalue())

### MVP company and country

In [31]:
role_df_tag1 = role_df_tag.drop_duplicates(subset=['mvp_company_name', 'country_data','publicLink'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
group_df4 = role_df_tag1.groupBy('mvp_company_name','country_data').count().orderBy('count',ascending = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
from io import StringIO # python3; python2: BytesIO import boto3
import boto3


bucket = 'draup-command-center' # already created on S3
csv_buffer = StringIO()

res2=group_df4.toPandas()
res2.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'temp/cc/Renukumar_Draup/karthika/output/pce_MVPcompany_cu_18-07-2024.csv').put(Body=csv_buffer.getvalue())

### GroupBy core skills and MVP company

In [14]:
skills_df = role_df_tag.withColumn('core_skills_data',explode(col('core_skills')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
group_df = skills_df.groupBy('Job role','Location','mvp_company_name','core_skills_data').count().orderBy('count',ascending = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
from pyspark.sql import Window
from pyspark.sql.functions import row_number
 
# Create a window specification
windowSpec = Window.partitionBy('Job role','Location','core_skills_data').orderBy(F.desc('count'))
 
# Add a row number column to each partition
grp_df10_with_row_num = group_df.withColumn('row_num', row_number().over(windowSpec))
 
# Filter to get only the top 10 rows per partition
final_df = grp_df10_with_row_num.filter(col('row_num') <= 20).drop('row_num')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
final_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Job role: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- mvp_company_name: string (nullable = true)
 |-- core_skills_data: string (nullable = true)
 |-- count: long (nullable = false)

In [26]:
from io import StringIO # python3; python2: BytesIO import boto3
import boto3


bucket = 'draup-command-center' # already created on S3
csv_buffer = StringIO()

res2=final_df.toPandas()
res2.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'temp/cc/Renukumar_Draup/karthika/output/pce_role_msa_MVP_skills_top_20_data_18-07-2024.csv').put(Body=csv_buffer.getvalue())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': '6PG101V3BEA1RP02', 'HostId': 'Cpp+h+jOhTDGR3O5SgiO/SbV4FjK8vAPl8QDrdfbTS9wqnwh65AXtXVb4MQZoEOAwvPGMcg8KyM=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'Cpp+h+jOhTDGR3O5SgiO/SbV4FjK8vAPl8QDrdfbTS9wqnwh65AXtXVb4MQZoEOAwvPGMcg8KyM=', 'x-amz-request-id': '6PG101V3BEA1RP02', 'date': 'Thu, 18 Jul 2024 11:44:36 GMT', 'x-amz-expiration': 'expiry-date="Wed, 24 Jul 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'x-amz-server-side-encryption': 'AES256', 'etag': '"961d5d0911151de11a067f13ae346340"', 'server': 'AmazonS3', 'content-length': '0'}, 'RetryAttempts': 1}, 'Expiration': 'expiry-date="Wed, 24 Jul 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'ETag': '"961d5d0911151de11a067f13ae346340"', 'ServerSideEncryption': 'AES256'}

### GroupBy Role,msa,MVP company

In [40]:
mvp_df = role_df_tag.groupBy('Job role','Location','mvp_company_name').count().orderBy('count',ascending = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
from pyspark.sql import Window
from pyspark.sql.functions import row_number
 
# Create a window specification
windowSpec = Window.partitionBy('Job role','Location','mvp_company_name').orderBy(F.desc('count'))
 
# Add a row number column to each partition
grp_df10_with_row_num = mvp_df.withColumn('row_num', row_number().over(windowSpec))
 
# Filter to get only the top 10 rows per partition
top_20_df = grp_df10_with_row_num.filter(col('row_num') <= 20).drop('row_num')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
from io import StringIO # python3; python2: BytesIO import boto3
import boto3


bucket = 'draup-command-center' # already created on S3
csv_buffer = StringIO()

res2=top_20_df.toPandas()
res2.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'temp/cc/Renukumar_Draup/karthika/output/pce_role_msa_MVP_top_20_data_18-07-2024.csv').put(Body=csv_buffer.getvalue())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': '24DAAKWNZK5CN3R2', 'HostId': 'Qq8dK0jf4cIO3e9xbuOq1kbsC51M1in86dmY5gNUDG5EjbT9tGAf2De0XPgwBUkZP0PFdLr6C8kcFNsZKpiBpg==', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'Qq8dK0jf4cIO3e9xbuOq1kbsC51M1in86dmY5gNUDG5EjbT9tGAf2De0XPgwBUkZP0PFdLr6C8kcFNsZKpiBpg==', 'x-amz-request-id': '24DAAKWNZK5CN3R2', 'date': 'Thu, 18 Jul 2024 10:33:17 GMT', 'x-amz-expiration': 'expiry-date="Wed, 24 Jul 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'x-amz-server-side-encryption': 'AES256', 'etag': '"8e24727d5ab68e400818de3ddd4f6f05"', 'server': 'AmazonS3', 'content-length': '0'}, 'RetryAttempts': 1}, 'Expiration': 'expiry-date="Wed, 24 Jul 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'ETag': '"8e24727d5ab68e400818de3ddd4f6f05"', 'ServerSideEncryption': 'AES256'}

### GroupBy Role,msa,MVP company, years of experience

In [34]:
expr_df = role_df_tag.groupBy('Job role','Location','mvp_company_name','years_of_experience').count().orderBy('count',ascending = False)
                     

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
from pyspark.sql import Window
from pyspark.sql.functions import row_number
 
# Create a window specification
windowSpec = Window.partitionBy('Job role','Location','mvp_company_name','years_of_experience').orderBy(F.desc('count'))
 
# Add a row number column to each partition
grp_df10_with_row_num = expr_df.withColumn('row_num', row_number().over(windowSpec))
 
# Filter to get only the top 10 rows per partition
top_20_df1 = grp_df10_with_row_num.filter(col('row_num') > 20).drop('row_num')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
from io import StringIO # python3; python2: BytesIO import boto3
import boto3


bucket = 'draup-command-center' # already created on S3
csv_buffer = StringIO()

res2=top_20_df1.toPandas()
res2.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'temp/cc/Renukumar_Draup/karthika/output/pce_role_msa_MVP_yearExprience_top_20_data_18-07-2024.csv').put(Body=csv_buffer.getvalue())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Interrupted by user
